In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data =pd.read_excel('dataset_train.xls',usecols=[1,8,9,10,11,12,13,14,17,18])

In [ ]:
data

# Target encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
LE=LabelEncoder()

In [ ]:
data['Hogwarts_House']=LE.fit_transform(data['Hogwarts House'])

In [ ]:
data.drop('Hogwarts House',axis=1,inplace=True)

In [ ]:
data

In [ ]:
data.isna().sum()

In [ ]:
for i in range(0,8):
    data[data.columns[i]].fillna(data[data.columns[i]].mean(),inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
data

## Normalizing inputs

In [ ]:
for i in range(9):
    x=data[data.columns[i]]   
    x=x-x.mean()
    x/=np.var(x)
    data[data.columns[i]]=x 

In [ ]:
data

# Create multiple datasets

In [ ]:
D0=data.copy();D1=data.copy();D2=data.copy();D3=data.copy() ## D0:Gryffindor // D1:Hufflepuff // D2:Ravenclaw // D3: Slytherin

In [ ]:
datasets=[D0,D1,D2,D3]

### This cell creates individual binary datasets

In [ ]:
for i in range(4):
    for j in range(1600):
        if datasets[i].at[j,'Hogwarts_House'] == i:
            datasets[i].at[j,'Hogwarts_House'] = 1
        else:
            datasets[i].at[j,'Hogwarts_House'] = 0

In [ ]:
D0

### dictionary holding dataset names as keys and their input and output splits as value pairs


In [20]:
datadict = {}  
for i in range(4):   
    X = datasets[i].iloc[:,:9]
    X = X.T
    Y = datasets[i].iloc[:,-1]
    Y = np.array([Y])
    datadict['D'+str(i)] = [X,Y] 
    

In [22]:
datadict['D0'][1]

array([[0, 0, 0, ..., 1, 0, 0]])

# 1 vs All classifier 

In [24]:
def sigmoid(x):
    sig = (1 / (1 + np.exp(-x)))     # Define sigmoid function
    return sig

In [25]:
itterations = 10000000   #random big value 
alpha = 0.160 
m = 1600
cost_values = []
trained_parameters = []
for trainsets in range(4):  
    X = datadict['D'+str(trainsets)][0] 
    y = datadict['D'+str(trainsets)][1]
    weights = np.random.randn(1,9)
    bias = 0 
    costfunc_values = []
    k = 0
    print('Training for dataset '+str(trainsets))
    for i in range(1,itterations+1):
        #logistic function
        z = np.dot(weights,X) + bias  
        hypothesis=sigmoid(z)
        
        #cost function
        j = 1/m*(-1*(np.sum(y*np.log(hypothesis) + (1-y)*np.log(1-hypothesis))))
        costfunc_values.append(j)
        k+=1

        #gradient decent
        dw =  1/m * np.dot(hypothesis-y,X.T)
        db =  1/m * np.sum(hypothesis-y)
        weights = weights - alpha*dw
        bias = bias - alpha*db

        #stop training
        if i%2000 == 0:
            print('running @ ',j)
        if i%2 == 0:
            if abs(j-costfunc_values[-2])<0.000001:
                if abs(j-costfunc_values[-3])<0.000001:
                    break 
    cost_values.append(costfunc_values)  
    trained_parameters.append([weights,bias])      
    print('itteration number:',k)


Training for dataset 0
running @  0.07922984684444988
running @  0.06581356208891465
running @  0.061760598859617315
running @  0.05986794245784648
itteration number: 9422
Training for dataset 1
running @  0.17756540365127088
running @  0.13187522071962096
running @  0.11431171812068655
running @  0.10513098190825332
running @  0.09953762202937796
running @  0.09579625949131185
running @  0.09312912558300003
running @  0.09113685691505737
running @  0.08959399946253076
running @  0.08836403437189726
itteration number: 21022
Training for dataset 2
running @  0.14587823127566982
running @  0.10806686563784396
running @  0.09529312774726738
running @  0.08921966590130825
running @  0.08579716090774646
running @  0.08365620335144083
running @  0.08221655320947295
itteration number: 15100
Training for dataset 3
running @  0.07502470883531316
running @  0.06274543012901616
running @  0.0592151080797346
running @  0.05765473950713492
itteration number: 8344


##  accuracy for each classifier

In [26]:
for datasetnum in range(4):
    X = datadict['D'+str(datasetnum)][0]
    y = datadict['D'+str(datasetnum)][1]
    weights = trained_parameters[datasetnum][0]
    bias = trained_parameters[datasetnum][1]
    correct_predictions = 0 
    for i in range(1600):
        z = np.dot(weights,X.T.iloc[i,:])+bias
        hypothesis = sigmoid(z)
        if np.logical_and(hypothesis >= 0.5,y.T[i,0] == 1):
            correct_predictions+=1
        if np.logical_and(hypothesis < 0.5,y.T[i,0] == 0):
            correct_predictions+=1          
    acc = (correct_predictions/1600)*100
    print('accuracy for dataset '+str(datasetnum)," = ",acc)

accuracy for dataset 0  =  98.75
accuracy for dataset 1  =  98.0
accuracy for dataset 2  =  98.625
accuracy for dataset 3  =  98.6875


## 1-VS-All classfier prediction accuracy 

In [27]:
inputs = data.iloc[:,:9]
inputs = inputs.T
outputs = data.iloc[:,-1] 
outputs = np.array(outputs)
accuratepredicts = 0                
for i in range(1600):
    probabilities = []
    for j in range(4):
        weights = trained_parameters[j][0]
        bias = trained_parameters[j][1]
        z = np.dot(weights,inputs.iloc[:,i].T)+bias
        hypothesis = sigmoid(z) 
        probabilities.append(hypothesis)
    predict = probabilities.index(max(probabilities)) 
    if outputs.T[i] == predict:
        accuratepredicts+=1
print((accuratepredicts/1600)*100)

97.75
